In [1]:
import json, random, os

# ===== Memory Components =====
class InMemorySessionService:
    def __init__(self): self.sessions = {}
    def get(self, sid): return self.sessions.setdefault(sid, {})
    def set(self, sid, data): self.sessions[sid] = data

class MemoryBank:
    def __init__(self, path="lunch_memory.json"): self.path = path
    def load(self): return json.load(open(self.path)) if os.path.exists(self.path) else {}
    def save(self, data): json.dump(data, open(self.path, "w"), indent=2)

# ===== Tools =====
def csv_tool():
    """Simulated CSV dataset of meals"""
    return [
        {"meal": "Chicken Salad", "calories": 450, "price": 7.5},
        {"meal": "Veggie Wrap", "calories": 380, "price": 6.0},
        {"meal": "Sushi Bento", "calories": 520, "price": 9.0},
        {"meal": "Beef Bowl", "calories": 640, "price": 8.5},
    ]

def openapi_tool(endpoint, params):
    """Mock external API to get meal rating"""
    if endpoint == "/rating":
        meal = params.get("meal", "Unknown")
        return {"meal": meal, "rating": round(random.uniform(3.5, 5.0), 1)}
    return {"error": "invalid endpoint"}

def code_execution_tool(data):
    """Run simple analysis"""
    avg_price = sum(m["price"] for m in data)/len(data)
    avg_cal = sum(m["calories"] for m in data)/len(data)
    return {"avg_price": round(avg_price,2), "avg_calories": round(avg_cal,2)}

# ===== Agents =====
class Planner:
    def run(self, goal): 
        return f"Plan: find healthy meals under ${goal['budget']} and below {goal['calories']} calories."

class Researcher:
    def run(self): 
        data = csv_tool()
        rated = []
        for d in data:
            d["rating"] = openapi_tool("/rating", {"meal": d["meal"]})["rating"]
            rated.append(d)
        return rated

class Coder:
    def run(self, meals): 
        return code_execution_tool(meals)

class Critic:
    def run(self, summary, goal):
        decision = "✅ Meets goal!" if summary["avg_price"]<=goal["budget"] and summary["avg_calories"]<=goal["calories"] else "⚠️ Needs adjustment."
        return decision

class Orchestrator:
    def __init__(self):
        self.session = InMemorySessionService()
        self.memory = MemoryBank()
        self.state = self.memory.load() or {}

    def run(self, goal):
        print("=== Healthy Lunch Planner AI ===")
        planner = Planner(); researcher = Researcher(); coder = Coder(); critic = Critic()

        print("\n📝 Planner:")
        plan = planner.run(goal); print(plan)

        print("\n🔍 Researcher:")
        meals = researcher.run()
        for m in meals: print(m)

        print("\n💻 Coder:")
        summary = coder.run(meals)
        print(summary)

        print("\n🧩 Critic:")
        decision = critic.run(summary, goal)
        print(decision)

        self.state = {"goal": goal, "plan": plan, "summary": summary, "decision": decision}
        self.memory.save(self.state)
        print("\n💾 Memory saved to lunch_memory.json")

# ===== Run the Orchestrator =====
goal = {"budget": 8.0, "calories": 600}
Orchestrator().run(goal)


=== Healthy Lunch Planner AI ===

📝 Planner:
Plan: find healthy meals under $8.0 and below 600 calories.

🔍 Researcher:
{'meal': 'Chicken Salad', 'calories': 450, 'price': 7.5, 'rating': 4.3}
{'meal': 'Veggie Wrap', 'calories': 380, 'price': 6.0, 'rating': 4.2}
{'meal': 'Sushi Bento', 'calories': 520, 'price': 9.0, 'rating': 4.7}
{'meal': 'Beef Bowl', 'calories': 640, 'price': 8.5, 'rating': 3.7}

💻 Coder:
{'avg_price': 7.75, 'avg_calories': 497.5}

🧩 Critic:
✅ Meets goal!

💾 Memory saved to lunch_memory.json
